In [ ]:
import math

TOTAL_DATA = 23527
KFOLD = 3
CONFIDENCE_LEVEL = 0.95

print(f"Nuestro total de datos es {TOTAL_DATA} y elegimos {KFOLD} para nuestro kfold con un nivel de confianza de {CONFIDENCE_LEVEL*100}%, lo que nos da lo siguiente:\n")

def calcular_error(n, confidence_level):
    return math.sqrt((math.log(2) - math.log(1 - confidence_level))/(2*n))

n_test = TOTAL_DATA / KFOLD
test_error = calcular_error(n_test, CONFIDENCE_LEVEL)
print(f"El error de testeo es= {test_error:.4f}\n")

n_train = n_test * 3
train_error = calcular_error(n_train, CONFIDENCE_LEVEL)
print(f"El error de entrenamiento es= {train_error:.4f}\n")

Nuestro total de datos es 23527 y elegimos 3 para nuestro kfold con un nivel de confianza de 95.0%, lo que nos da lo siguiente:

El error de testeo es= 0.0153

El error de entrenamiento es= 0.0089



In [ ]:
import pandas as pd
import numpy as np
import random

# Cargar datos
df = pd.read_csv("factVentasEtiquetados.csv", encoding='latin-1', delimiter=';')

# Función para obtener etiquetas únicas
def obtener_etiquetas_unicas(df, columna_etiqueta):
    etiquetas = df[columna_etiqueta]
    print("Etiquetas únicas:", etiquetas.unique() if etiquetas is not None else "No se encontraron etiquetas")
    return etiquetas.unique()

# Función para segmentar datos en matrices
def segmentar_en_matrices(df, etiqueta):
    etiquetas = obtener_etiquetas_unicas(df, etiqueta)
    print("Etiquetas únicas:", etiquetas)
    matrices = {}
    for etiqueta_unica in etiquetas:
        matriz = df[df[etiqueta] == etiqueta_unica]
        matrices[etiqueta_unica] = matriz
    return matrices

# Segmentar los datos en matrices
matrices = segmentar_en_matrices(df, 'etiqueta')

# Asignar matrices de manera segura verificando si las etiquetas existen
matriz1 = matrices.get(1)
matriz2 = matrices.get(2)
matriz3 = matrices.get(3)

# Dividir en conjuntos, manejando casos donde la matriz puede ser None
def dividir_en_conjuntos(matriz):
    if matriz is not None and not matriz.empty:
        num_filas = len(matriz)
        tamano_conjunto = num_filas // 3
        resto = num_filas % 3
        conjuntos = []
        inicio = 0
        for i in range(3):
            fin = inicio + tamano_conjunto
            if resto > 0:
                fin += 1
                resto -= 1
            conjuntos.append(matriz.iloc[inicio:fin])
            inicio = fin
        return conjuntos
    else:
        return [None] * 7

# Dividir en conjuntos para cada matriz
conjuntos1 = dividir_en_conjuntos(matriz1)
conjuntos2 = dividir_en_conjuntos(matriz2)
conjuntos3 = dividir_en_conjuntos(matriz3)

# Concatenar conjuntos válidos
def concatenate_valid_sets(sets):
    non_empty_sets = [s for s in sets if s is not None]
    if non_empty_sets:
        max_rows = max(s.shape[0] for s in non_empty_sets)
        padded_sets = [np.pad(s.values, ((0, max_rows - s.shape[0]), (0, 0)), mode='constant') for s in non_empty_sets]
        concatenated_sets = np.concatenate(padded_sets, axis=1)
        return concatenated_sets
    else:
        # Handle the case where all sets are empty
        print("Warning: All input sets are empty. Returning an empty array.")
        return np.array([])

# Concatenar conjuntos válidos para cada test
K1 = concatenate_valid_sets(conjuntos1)
K2 = concatenate_valid_sets(conjuntos2)
K3 = concatenate_valid_sets(conjuntos3)

# Mezclar los datos de cada conjunto
def mezclar_matriz(matriz):
    # Crea una copia de la matriz original para no modificarla
    resultado = matriz.copy()
    # Mezcla las filas de la matriz
    np.random.shuffle(resultado)
    return resultado

K1_Shuffle = mezclar_matriz(K1)
K2_Shuffle = mezclar_matriz(K2)
K3_Shuffle = mezclar_matriz(K3)

# Guardar conjuntos en archivos CSV
for i, K_Shuffle in enumerate([K1_Shuffle, K2_Shuffle, K3_Shuffle]):
    pd.DataFrame(K_Shuffle).to_csv(f"Test{i+1}.csv", index=False)

# Calcular el total de filas
total_rows = min(K1_Shuffle.shape[0], K2_Shuffle.shape[0], K3_Shuffle.shape[0])

# Generar una lista de índices aleatorios
random_indices = list(range(total_rows))
random.shuffle(random_indices)

# Truncar los índices aleatorios para asegurarse de que estén dentro de los límites de la matriz
random_indices = [idx % total_rows for idx in random_indices]

# Seleccionar las filas correspondientes usando los índices aleatorios
K1_Shuffle = K1_Shuffle[random_indices]
K2_Shuffle = K2_Shuffle[random_indices]
K3_Shuffle = K3_Shuffle[random_indices]

# Dividir conjuntos para entrenamiento
Training1 = pd.DataFrame(np.concatenate((K2_Shuffle, K3_Shuffle), axis=0))
Training2 = pd.DataFrame(np.concatenate((K3_Shuffle, K1_Shuffle), axis=0))
Training3 = pd.DataFrame(np.concatenate((K1_Shuffle, K2_Shuffle), axis=0))

# Guardar conjuntos de entrenamiento en archivos CSV
Training1.to_csv("Training1.csv", index=False)
Training2.to_csv("Training2.csv", index=False)
Training3.to_csv("Training3.csv", index=False)


Etiquetas únicas: [0 1 2 3]
Etiquetas únicas: [0 1 2 3]
